In [5]:
#Importing the required libraries
import os
import glob
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import fftpack
from scipy.stats import kurtosis,skew,mode
import sklearn.preprocessing,sklearn.decomposition
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,train_test_split
from keras import utils
import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv1D, Conv2D, Flatten,Reshape, BatchNormalization, ZeroPadding2D,MaxPooling1D,AveragePooling1D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras import regularizers,optimizers
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
import keras.backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

In [6]:
import os

def get_audio_files(root_directory):
    """
    Traverse folder structure to find all .ogg files and assign labels based on folder names.
    
    Parameters:
        root_directory (str): Path to the root directory containing language folders.
    
    Returns:
        list: A list of dictionaries with file paths and their corresponding labels.
    """
    data = []
    
    for folder in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder)
        if os.path.isdir(folder_path):  # Ensure it's a folder
            language_label = folder.replace("SWC_", "")  # Extract language from folder name
            
            # Recursively traverse subfolders
            for subfolder, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith(".ogg"):  # Only process .ogg files
                        file_path = os.path.join(subfolder, file)
                        print(f"Found file: {file_path}, Label: {language_label}")
                        data.append({"file_path": file_path, "label": language_label})
    return data

# Example usage
root_directory = r"C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC"
audio_data = get_audio_files(root_directory)
print(f"Total files found: {len(audio_data)}")


Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27s-Heerenberg\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Beijersche\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Goy\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Haantje_(Rijswijk)\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Harde\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Heem_(Ter_Apel)\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\15\audio.ogg, Label: Dutch
Found file: C:\Users\pavan\Documents\Fall 2024\EE502 

In [7]:
import librosa
import numpy as np

def extract_features(file_path, duration=20):
    """
    Extract audio features (MFCC, Chroma, Spectral Contrast) from a given audio file.
    
    Parameters:
        file_path (str): Path to the audio file.
        duration (int): Duration to load in seconds (default is 240 seconds).
    
    Returns:
        np.array: Combined feature array (MFCC, Chroma, Spectral Contrast).
    """
    try:
        # Load audio file (only the first `duration` seconds)
        audio, sr = librosa.load(file_path, sr=22050, duration=duration)
        
        # Extract features
        mfcc = librosa.feature.mfcc(y=audio, n_fft = 441, hop_length = 220, sr=sr, n_mfcc=13)
        chroma = librosa.feature.chroma_stft(y=audio, n_fft = 441, hop_length = 220, sr=sr)
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, n_fft = 441, hop_length = 220, sr=sr)
        
        # Compute means
        mfcc_mean = np.mean(mfcc, axis=1)
        chroma_mean = np.mean(chroma, axis=1)
        spectral_contrast_mean = np.mean(spectral_contrast, axis=1)
        
        # Combine features
        return np.hstack([mfcc_mean, chroma_mean, spectral_contrast_mean])
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None



In [8]:
def create_dataset(audio_data, duration=20):
    """
    Create a dataset by extracting features from all audio files.
    
    Parameters:
        audio_data (list): List of dictionaries containing file paths and labels.
        duration (int): Duration to load for each file in seconds.
    
    Returns:
        list: A list of rows containing features and labels.
    """
    dataset = []
    for item in audio_data:
        file_path = item["file_path"]
        label = item["label"]
        
        # Extract features
        features = extract_features(file_path, duration=duration)
        if features is not None:
            dataset.append([*features, label])
    return dataset

# Example usage
dataset = create_dataset(audio_data, duration=20)
print(f"Total processed files: {len(dataset)}")


Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Blaaksedijk\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Burundi_women%27s_national_football_team\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Canada_men%27s_national_softball_team\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Casey_Samuels\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Chinese_Taipei_men%27s_national_softball_team\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_English\english\Christie_Dawes\audio.ogg: 
Error processing file C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_E

In [9]:
import pandas as pd

def save_to_csv(dataset, output_path):
    """
    Save the dataset to a CSV file.
    
    Parameters:
        dataset (list): List of rows containing features and labels.
        output_path (str): Path to save the CSV file.
    """
    # Define column names
    columns = [f'MFCC_{i}' for i in range(13)] + \
              [f'Chroma_{i}' for i in range(12)] + \
              [f'SpectralContrast_{i}' for i in range(7)] + ['Label']
    
    # Create a DataFrame
    df = pd.DataFrame(dataset, columns=columns)
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"Dataset saved to {output_path}")

# Example usage
output_csv = r"C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\language_features.csv"
save_to_csv(dataset, output_csv)


Dataset saved to C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\language_features.csv


In [13]:
import librosa
import noisereduce as nr
import numpy as np

def reduce_noise(y, sr, noise_reduction_factor=0.1, n_iter=3):
    """
    Apply noise reduction to an audio signal.
    
    Parameters:
        y (np.array): The audio signal.
        sr (int): Sampling rate of the audio.
        noise_reduction_factor (float): Threshold multiplier for noise reduction.
        n_iter (int): Number of iterations for noise reduction.

    Returns:
        np.array: The denoised audio signal.
    """
    for _ in range(n_iter):
        # Compute the STFT of the audio signal
        stft = librosa.stft(y)
        stft_magnitude, stft_phase = librosa.magphase(stft)

        # Estimate noise profile from the first 1 second
        noise_magnitude = np.mean(stft_magnitude[:, :int(sr * 0.1)], axis=1)
        
        # Create a mask to attenuate regions below the noise threshold
        mask = stft_magnitude > noise_reduction_factor * noise_magnitude[:, np.newaxis]
        
        # Apply the mask to the magnitude spectrogram
        stft_magnitude_denoised = stft_magnitude * mask
        
        # Inverse STFT to reconstruct the denoised signal
        stft_denoised = stft_magnitude_denoised * stft_phase
        y = librosa.istft(stft_denoised)
    
    return y


In [14]:
import librosa
import numpy as np

def extract_features_denoised(file_path, duration=20, noise_reduction_factor=10, n_iter=10):
    """
    Extract features from an audio file with noise reduction.

    Parameters:
        file_path (str): Path to the audio file.
        duration (int): Duration to load for the file in seconds.
        noise_reduction_factor (float): Threshold multiplier for noise reduction.
        n_iter (int): Number of iterations for noise reduction.

    Returns:
        list: Extracted features or None if an error occurs.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(file_path, sr = 22050, duration=duration)

        # Apply noise reduction
        noise_profile = np.percentile(np.abs(y), noise_reduction_factor)
        for _ in range(n_iter):
            y = np.where(np.abs(y) < noise_profile, 0, y)

        # Extract features
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_fft = 441, hop_length = 220, n_mfcc=13).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr, n_fft = 441, hop_length = 220).T, axis=0)

        # Combine features into a single array
        features = np.hstack([mfccs, chroma, spectral_contrast])
        return features

    except Exception as e:
        print(f"Error extracting features from file {file_path}: {e}")
        return None


In [15]:
def save_dataset_to_csv(dataset, output_path):
    """
    Save the dataset to a CSV file.
    
    Parameters:
        dataset (pd.DataFrame): DataFrame containing features and labels.
        output_path (str): Path to save the CSV file.
    """
    dataset.to_csv(output_path, index=False)
    print(f"Dataset saved to {output_path}")


In [16]:
# Step 1: Get audio files and labels (if not already done)
# audio_data = get_audio_files(root_directory)  # Use this if `audio_data` is not defined

import pandas as pd

# Step 2: Extract features from denoised audio
denoised_dataset = create_denoised_feature_dataset_from_audio_data(audio_data, duration=20, noise_reduction_factor=10, n_iter=10)

# Step 3: Save the dataset to a CSV file
output_csv = r"C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\denoised_features.csv"
save_dataset_to_csv(denoised_dataset, output_csv)


Processing file 1/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27s-Heerenberg\audio.ogg
Processing file 2/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Beijersche\audio.ogg
Processing file 3/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Goy\audio.ogg
Processing file 4/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Haantje_(Rijswijk)\audio.ogg
Processing file 5/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Harde\audio.ogg
Processing file 6/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\%27t_Heem_(Ter_Apel)\audio.ogg
Processing file 7/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML in Bio\Final Project\SWC\SWC_Dutch\dutch\15\audio.ogg
Processing file 8/2691: C:\Users\pavan\Documents\Fall 2024\EE502 ML